In [1]:
input_path='../inputs/230105_recipes.xlsx'
output_dir_path='../htmls/'
output_path='../index.html'

In [2]:
from pathlib import Path
Path(output_dir_path).mkdir(exist_ok=True)

In [3]:
import pandas as pd
sheet_names = sorted(list(set(pd.ExcelFile(input_path).sheet_names) - set(['template'])))
sheet_names

['Breakfast-Pancakes',
 'Meal-Lasagne',
 'Meal-Pizza dough',
 'Meal-Pumpkin soup',
 'Snacks-Banana muffins',
 'Snacks-Cookies',
 'Snacks-Scones']

In [4]:
sections=pd.Series(dict(zip(sheet_names,[s.split('-')[0] for s in sheet_names]))).reset_index().groupby(0)['index'].agg(list).to_dict()
sections

{'Breakfast': ['Breakfast-Pancakes'],
 'Meal': ['Meal-Lasagne', 'Meal-Pizza dough', 'Meal-Pumpkin soup'],
 'Snacks': ['Snacks-Banana muffins', 'Snacks-Cookies', 'Snacks-Scones']}

In [5]:
from xlsx2html import xlsx2html
from bs4 import BeautifulSoup
text_sections=[]
for section in sections:
    text_tables=[]
    for k in sections[section]:
        xlsx2html(input_path, f"{output_dir_path}/{k}.html",sheet=k)
        ## title
        text_tables+=[f'<button type="button" class="collapsible">'+k.split('-')[1]+'</button>']
        with open(f"{output_dir_path}/{k}.html") as fp:
            _table=str(BeautifulSoup(fp, 'html.parser').find('table'))
        text_tables.append('<div class="content">'+_table.replace("<table",'<table class="recipe_card"')+'</div>')
        # break
    text_sections.append(f'<button type="button" class="collapsible">{section}</button><div class="content">'+(''.join(text_tables))+'</div>')

In [6]:
text_page=("""<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Cooking For Engineers</title>
    <link href="css/global.css" rel="stylesheet"/>
    <link href="css/sections.css" rel="stylesheet"/>
    <link rel="stylesheet" href="https://fonts.googleapis.com/css?family=Share+Tech+Mono">
    <link href="https://fonts.googleapis.com/css?family=Lobster" rel="stylesheet"/>
</head>
<body>
  <div class="container">
      <div class="title">
        <h1 style="font-size:xx-large;text-align: center;font-family: Lobster !important;">🍳 Cooking for engineers 👷🏻‍♀️</h1>
      </div>
      <div class="scale">
       <label for="multiplier">Select a multiplier:</label>
       <select id="multiplier" onchange="multiplyValues()">
         <option value="0.5">0.5x</option>
         <option selected value="1">1x</option>
         <option value="2">2x</option>
         <option value="3">3x</option>
         <option value="5">5x</option>
       </select>
      </div>
      <div>
        <span id="scale_status">Status: 1x</span><button onClick="window.location.reload();">Reset</button>
      </div>
      <div id="recipes" class="recipes">
"""
+''.join(text_sections)
+
"""
    </div>
</div>
    <script src="js/sections.js"></script>
    <script src="js/math.js"></script>
</body>
</html>""")

In [7]:
soup = BeautifulSoup(text_page, 'html.parser')

In [8]:
with open(output_path,'w') as f:
    f.write(soup.prettify())